In [10]:
import aes2_added_fb_prize_as_features_preprocessing

In [11]:
import gc

gc.collect()

584

In [12]:
import pickle
import sys
sys.path.append('/home/mcq/GitHub/aes2')
sys.path.append('/home/mcq/GitHub/aes2/topic-classificatio')
import numpy as np
from sklearn.model_selection import GroupKFold
import pandas as pd
import pickle
with open("/home/mcq/GitHub/aes2/train_data/train_feats.pkl", "rb") as f:
    train_feats = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/X.pkl", "rb") as f:
    X = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/y.pkl", "rb") as f:
    y = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/y_split.pkl", "rb") as f:
    y_split = pickle.load(f)
with open(
    "/home/mcq/GitHub/aes2/train_data/feature_select.pkl", "rb"
) as f:
    feature_select = pickle.load(f)
with open(
    "/home/mcq/GitHub/aes2/train_data/base_skf_feat/category-feat.pkl", "rb"
) as f:
    topic = pickle.load(f)
    
aes2_added_fb_prize_as_features_preprocessing.feature_select = feature_select



In [13]:
train_feats.iloc[:5, -20:]
train_feats_1 = train_feats[train_feats['essay_id'].str.len() == 12] #筛选额外数据集的feats
train_feats_2 = train_feats[train_feats['essay_id'].str.len() == 7] #筛选原数据集的feats
print(len(train_feats_1.shape))
print(len(train_feats_2.shape))

2
2


In [14]:
X.shape

(30178, 13000)

In [15]:
import numpy as np

def clean_feature_names(features):
    illegal_chars = ['[', ']', '<', '>']
    cleaned_features = []
    for feature in features:
        for char in illegal_chars:
            feature = feature.replace(char, 'lessthan')
        cleaned_features.append(feature)
    return cleaned_features

train_feats.columns = clean_feature_names(train_feats.columns)
feature_select = clean_feature_names(feature_select)
with open("/home/mcq/GitHub/aes2/train_data/train_2.pkl", "rb") as f:
    train_2 = pickle.load(f)

with open("/home/mcq/GitHub/aes2/train_data/GKF_train.pkl", "rb") as f:
    GKF_train = pickle.load(f)

X = train_feats[feature_select].astype(np.float32).values
y_split = train_2['score'].astype(np.float32).values 

X_2 = train_feats_1[feature_select].astype(np.float32).values ##注意这里的X2要更改为GKF_train 的feature select 之后的文件
#X_2 = train_feats[feature_names].astype(np.float32).values 
y_split_2 = GKF_train['score'].astype(int).values



# Find Threshold

In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, cohen_kappa_score
from aes2_added_fb_prize_as_features_preprocessing import *



def find_thresholds(true, pred, steps=50):

    # SAVE TRIALS FOR PLOTTING
    xs = [[],[],[],[],[]]
    ys = [[],[],[],[],[]]

    # COMPUTE BASELINE METRIC
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")

    # FIND FIVE OPTIMAL THRESHOLDS
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold2 = threshold.copy()
            stop = 0
            while stop<steps:

                # TRY NEW THRESHOLD
                v += sign * 0.01
                threshold2[k] = v
                pred2 = pd.cut(pred, [-np.inf] + threshold2 + [np.inf], 
                                labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred2, weights="quadratic")

                # SAVE TRIALS FOR PLOTTING
                xs[k].append(v)
                ys[k].append(metric)

                # EARLY STOPPING
                if metric<=best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold2.copy()

    # COMPUTE FINAL METRIC
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")   

    # RETURN RESULTS
    threshold = [np.round(t,3) for t in threshold]
    return best, threshold, xs, ys

# best, thresholds, xs, ys = find_thresholds(y_split, oof, steps=500)
# print('Best thresholds are:', thresholds )
# print('=> achieve Overall CV QWK score =', best )

# >> Output: thresholds = [1.5, 2.6, 3.5, 4.59, 5.55]
# thresholds = [1.51, 2.6, 3.5, 4.59, 5.56]

# skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=0)
# train_index_2 = []
# i=1
# for train_index, test_index in skf.split(X_2,y_split_2):
#     train_index_2.append(train_index)





models = []
predictions = []
f1_scores = []
kappa_scores = []
groups = topic['topic']
tuner_params = {
    "gpu_id" : 7,
    "learning_rate_lgb": 0.05,
    "max_depth_lgb": 8,
    "num_leaves_lgb": 10,
    "colsample_bytree_lgb": 0.3,
    "reg_alpha_lgb": 0.7,
    "reg_lambda_lgb": 0.1,
    "n_estimators_lgb": 700,
    "learning_rate_xgb": 0.1,
    "max_depth_xgb": 8,
    "num_leaves_xgb": 10,
    "colsample_bytree_xgb": 0.5,
    "reg_alpha_xgb": 0.1,
    "reg_lambda_xgb": 0.8,
    "n_estimators_xgb": 1024,
    "n_splits": 7
}

n_splits = tuner_params['n_splits']
oof = np.zeros(len(train), dtype='float32')

class Predictor:
    def __init__(self, models: list):
        self.models = models
#         self.xgb_boost_best_iter = models[1].
    def predict(self, X):
        n_models = len(self.models)
        predicted = None
        # n = 0.709
        n = 0.749
        for i, model in enumerate(self.models):
            if i == 0:
                predicted = n*model.predict(X)
            else:
                # if not isinstance(X, xgb.DMatrix):
                #     X = xgb.DMatrix(X)
                predicted += (1-n)*model.predict(X)
        return predicted

group_kfold = GroupKFold(n_splits=n_splits)
xgboost_best_iters = []
light_best_iters = []
# thresholds = [1.5, 2.5, 3.5, 4.5, 5.5] # init threshold


LOAD = True # re-train
if not LOAD:
    for i in range(n_splits):
        random_indices = np.random.choice(X.shape[0], size=5000, replace=False)
        random_test_X = X[random_indices, :]
        random_test_y = y_split[random_indices]
        lgb_predictor = lgb.Booster(model_file=f'kaggle/input/3models/aes-lgbm/fold_{i+1}.txt')
        # cat_model = CatBoostRegressor()
        # cat_model.load_model(f'kaggle/input/aes-catboost/fold_{i+1}.cbm')
        xgb_regressor = xgb.Booster()
        xgb_regressor.load_model(f'kaggle/input/3models/aes-xgboost/fold_{i+1}.bin')

        predictor = Predictor([lgb_predictor, xgb_regressor])
        
        models.append(predictor)
        
        # # Make predictions on the test data for this fold
        #         # Make predictions on the test data for this fold
        predictions_fold = predictor.predict(random_test_X)
        predictions_fold = predictions_fold + a
        predictions_fold = predictions_fold.clip(1, 6).round().astype(int)
        
        # # Confusion Matrix for each fold
        # cm = confusion_matrix(random_test_y, predictions_fold, labels=[x for x in range(1,7)])
        # disp = ConfusionMatrixDisplay(confusion_matrix=cm,
        #                               display_labels=[x for x in range(1,7)])
        # disp.plot()
        # plt.show()
        
        # # Calculate and store the F1 score for this fold
        f1_fold = f1_score(random_test_y, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

        # Calculate and store the Cohen's kappa score for this fold
        kappa_fold = cohen_kappa_score(random_test_y, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)

        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')
        gc.collect()
else:
    for i, (train_index, test_index) in enumerate(group_kfold.split(X, y_split,groups), 1):
    # Split the data into training and testing sets for this fold
        print('fold',i)
        # X_train_fold, X_test_fold = X[train_index], X[test_index]
        # y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]
        
        
        X_train_fold_1, X_test_fold = X[train_index], X[test_index]
        X_train_fold_2 = X_2[train_index_2[i]]
        X_train_fold = y_train_fold = np.hstack((X_train_fold_1, X_train_fold_2))
        ## 链接2个训练集

        y_train_fold_1, y_test_fold, y_test_fold_int = y_split[train_index], y_split[test_index], y_split[test_index]
        y_train_fold_2 = y_split_2[train_index_2[i]]
        y_train_fold = np.hstack((y_train_fold_1, y_train_fold_2))
        
        
        
        callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75,first_metric_only=True)]
        light = lgb.LGBMRegressor(
                objective = qwk_obj,
                metrics = 'None',
                learning_rate = tuner_params["learning_rate_lgb"],
                max_depth = tuner_params["max_depth_lgb"],
                num_leaves = tuner_params["num_leaves_lgb"],
                colsample_bytree = tuner_params["colsample_bytree_lgb"],
                reg_alpha = tuner_params["reg_alpha_lgb"],
                reg_lambda = tuner_params["reg_lambda_lgb"],
                n_estimators = tuner_params["n_estimators_lgb"],
                random_state=42,
                extra_trees=True,
                class_weight='balanced',
                # device='gpu' if CUDA_AVAILABLE else 'cpu',
                # device_type="gpu",
                verbosity = - 1
            )

        # Fit the model on the training data for this fold  
        light.fit(
            X_train_fold,
            y_train_fold,
            eval_names=['train', 'valid'],
            eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
            eval_metric=quadratic_weighted_kappa,
            callbacks=callbacks
        )
        light_best_iters.append(light.best_iteration_)
        
        
        light.booster_.save_model(f'kaggle/out/aes-lgbm/fold_{i}.txt')
        print('\nFold_{} LightGBM Model saved.\n'.format(i))
        
        
        xgb_callbacks = [
            xgb.callback.EvaluationMonitor(period=25),
            xgb.callback.EarlyStopping(75, metric_name="QWK", maximize=True, save_best=True)
        ]
        xgb_regressor = xgb.XGBRegressor(
            objective = qwk_obj,
            metrics = 'None',
            learning_rate = tuner_params["learning_rate_xgb"],
            max_depth = tuner_params["max_depth_xgb"],
            num_leaves = tuner_params["num_leaves_xgb"],
            colsample_bytree = tuner_params["colsample_bytree_xgb"],
            reg_alpha = tuner_params["reg_alpha_xgb"],
            reg_lambda = tuner_params["reg_lambda_xgb"],
            n_estimators = tuner_params["n_estimators_xgb"],
            random_state=42,
            extra_trees=True,
            # scale_pos_weight=100,
            class_weight='balanced',
            tree_method="hist",
            # device="gpu" if CUDA_AVAILABLE else "cpu",
            gpu_id = tuner_params["gpu_id"]
        #             device='gpu',
        #             verbosity = 1
        )
        
        xgb_callbacks = [
            xgb.callback.EvaluationMonitor(period=25),
            xgb.callback.EarlyStopping(75, metric_name="QWK", maximize=True, save_best=True)
        ]
        xgb_regressor.fit(
            X_train_fold,
            y_train_fold,
            eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
            eval_metric=quadratic_weighted_kappa,
            callbacks=xgb_callbacks
        )
        xgboost_best_iters.append(xgb_regressor.get_booster().best_iteration)
        
        xgb_regressor.save_model(f'kaggle/out/aes-xgboost/fold_{i}.bin')
        print('\nFold_{} XGBoost Model saved.\n'.format(i)) 
        
        predictor = Predictor([light, xgb_regressor])
        
        models.append(predictor)
        # Make predictions on the test data for this fold
        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        
        
        # predictions_fold = pd.cut(
        #     predictions_fold, [-np.inf] + thresholds + [np.inf], 
        #     labels=[1,2,3,4,5,6]
        #     ).astype('int32')
        # oof[test_index] = predictions_fold
        
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        # Calculate and store the F1 score for this fold
        f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

        # Calculate and store the Cohen's kappa score for this fold
        kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)
    #         predictor.booster_.save_model(f'fold_{i}.txt')
        cm = confusion_matrix(y_test_fold_int, predictions_fold, labels=[x for x in range(1,7)])

        disp = ConfusionMatrixDisplay(
            confusion_matrix=cm,
            display_labels=[x for x in range(1,7)]
        )
        disp.plot()
        plt.show()
        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')

        gc.collect()
        #if ENABLE_DONT_WASTE_YOUR_RUN_TIME:
        #    break


ValueError: Found input variables with inconsistent numbers of samples: [12819, 13121]

In [ ]:
mean_f1_score = np.mean(f1_scores)
mean_kappa_score = np.mean(kappa_scores)
# Print the mean scores
print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')
# print(f"XGBoost mean best iters: {sum(xgboost_best_iters)/len(xgboost_best_iters)}")
# print(f"LightBoost mean best iters: {sum(light_best_iters)/len(light_best_iters)}")

Mean F1 score across 7 folds: 0.6581823825289266
Mean Cohen kappa score across 7 folds: 0.8039929789959077


In [ ]:

# from aes2_added_fb_prize_as_features_preprocessing import preprocess_test, infer
import aes2_added_fb_prize_as_features_preprocessing

test_feats = aes2_added_fb_prize_as_features_preprocessing.preprocess_test()
test_feats.columns = clean_feature_names(test_feats.columns)
aes2_added_fb_prize_as_features_preprocessing.infer(test_feats, models)


tokenizers.__version__: 0.19.1
transformers.__version__: 4.41.1


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 502.00 MiB. GPU 